In [1]:
import pymongo
from pymongo import MongoClient
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv
from sklearn import preprocessing

In [7]:
survey_records = pd.read_csv("2018-2-3-Cleaned_Survey_Timeseries_Data.csv", index_col = 0)

## Census Bureau-designated regions and divisions

U.S. Census Bureau Regions and Divisions.
The United States Census Bureau defines four statistical regions, with nine divisions.[1] The Census Bureau region definition is "widely used for data collection and analysis," and is the most commonly used classification system.

Regional divisions used by the United States Census Bureau:

#### Region 1: Northeast

 Connecticut, Maine, Massachusetts, New Hampshire, Rhode Island, Vermont ,New Jersey, New York, and Pennsylvania.
 

#### Region 2: Midwest (Prior to June 1984, the Midwest Region was designated as the North Central Region.)
Illinois, Indiana, Michigan, Ohio, and Wisconsin, Iowa, Kansas, Minnesota, Missouri, Nebraska, North Dakota, and South Dakota


#### Region 3: South
Delaware, Florida, Georgia, Maryland, North Carolina, South Carolina, Virginia, District of Columbia, West Virginia  Alabama, Kentucky, Mississipp, Tennessee, Arkansas, Louisiana, Oklahoma, and Texas



#### Region 4: West
Arizona, Colorado, Idaho, Montana, Nevada, New Mexico, Utah, Wyoming, Alaska, California, Hawaii, Oregon, and Washington
Puerto Rico and other US territories are not part of any census region or census division.




In [9]:
Northeast = ['09','23','25','33','44','50','34','36','42'] 
Midwest = ['17','18','21','39','55','19','20','27','29','31','38','46']
South = ['10','12','13','24','37','45','51','11','54','01','21','28','47','05','22','40','48']
West = ['04','08','16','32','30','35','49','56','01','06','15','41','53','72']

### COMPUTATION OF BASELINE DIFFERENCE SCORE 

In [13]:
############################## COMPUTATION OF BASELINE DIFFERENCE SCORE ########################

def getListOfAvgs(AgeFlatList, subsRatingsFlatList):
    """ returns average condition rating of the bridges at a age"""
    sum_by_age = {}
    counts_of_aba = {}
    
    for i in range(0,150,1):
        sum_by_age[i] = 0
        counts_of_aba[i] = 0


    for i,j in zip(subsRatingsFlatList,AgeFlatList):
        for rating, age in zip(i,j):
            try:
                counts_of_aba[age]= counts_of_aba[age] + 1
                sum_by_age[age] = sum_by_age[int(age)] + int(rating)
            except:
                pass

    list_of_avgs = [sums/count for sums, count in zip(sum_by_age.values(), counts_of_aba.values()) if count !=0 ]
    ages = [age for age in range(1,101,1)]
    dict_of_avgs = {age:avg for age, avg in zip(ages,list_of_avgs)}
    return list_of_avgs, dict_of_avgs, counts_of_aba


def computeBaselineScore(ages, condition_ratings, dict_of_avgs):
    """Computation of Baseline Score"""
    scores_temp = []
    for ratings, age in zip(condition_ratings, ages):
        temp_list = []
        for rating, a in zip(ratings,age):
            try:
                temp_list.append(int(rating) -  dict_of_avgs[a])
            except:
                pass
        scores_temp.append(temp_list)
    scores = [np.mean(score) for score in scores_temp]
    return scores



## Northeast

In [14]:
survey_records_northeast = survey_records[survey_records['State Code'].isin(Northeast)]
age_flat_list_northeast = survey_records_northeast['Age'].tolist()

superstructure_flat_list_northeast = survey_records_northeast['Superstructure'].tolist()

list_of_avgs_northeast, dict_of_avgs_northeast, counts_of_aba_northeast = getListOfAvgs(age_flat_list_northeast, superstructure_flat_list_northeast)

baselineScores_northeast = computeBaselineScore(age_flat_list_northeast, superstructure_flat_list_northeast, dict_of_avgs_northeast)


survey_records_northeast['Baseline Difference Score'] = baselineScores_northeast
survey_records_northeast = survey_records_northeast.dropna()

#survey_records_northeast['Scaled Baseline Difference Score'] =  preprocessing.scale(survey_records_northeast['Baseline Difference Score'])

/Users/AkshayKale/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/AkshayKale/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/AkshayKale/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required by the scale function.

In [17]:
random samples

In [18]:
survey_records_northeast

,Structure Number,Age,ADT Type,Superstructure,Avg. Daily Precipitation,State Code,Deterioration Score
32770,000000000000001_1,"[48, 49, 50, 51, 52, 53, 54]","['Light', 'Light', 'Light', 'Light', 'Light', ...","[7, 7, 7, 7, 7, 7, 7]","[3.27, 3.27, 3.27, 3.27, 3.27, 3.27, 3.27]",42,0.000000
32772,000000000000002_2,"[21, 22, 23, 24, 25, 26, 27]","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","[7, 7, 7, 7, 7, 7, 7]","[3.27, 3.27, 3.27, 3.27, 3.27, 3.27, 3.27]",42,0.000000
32773,000000000000003_1,"[48, 49, 50, 51, 52, 53, 54]","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","[6, 6, 6, 6, 6, 6, 6]","[3.27, 3.27, 3.27, 3.27, 3.27, 3.27, 3.27]",42,0.000000
32774,000000000000004_1,"[21, 22, 23, 24, 25, 26, 27]","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","[8, 8, 8, 8, 8, 8, 8]","[3.27, 3.27, 3.27, 3.27, 3.27, 3.27, 3.27]",42,0.000000
32776,000000000000005_1,"[48, 49, 50, 51, 52, 53, 54]","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","[8, 8, 8, 8, 8, 8, 8]","[3.27, 3.27, 3.27, 3.27, 3.27, 3.27, 3.27]",42,0.000000
32779,000000000000008_2,"[21, 22, 23, 24, 25, 26, 27]","['Light', 'Light', 'Light', 'Light', 'Light', ...","[7, 7, 7, 7, 7, 7, 7]","[3.27, 3.27, 3.27, 3.27, 3.27, 3.27, 3.27]",42,0.000000
32785,000000000000010_2,"[48, 49, 50, 51, 52, 53, 54]","['Light', 'Light', 'Light', 'Light', 'Light', ...","[7, 7, 7, 7, 7, 7, 7]","[3.27, 3.27, 3.27, 3.27, 3.27, 3.27, 3.27]",42,0.000000
32788,000000000000012_2,"[21, 22, 23, 24, 25, 26, 27]","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","[8, 8, 8, 8, 8, 7, 7]","[3.27, 3.27, 3.27, 3.27, 3.27, 3.27, 3.27]",42,-0.166667
32789,000000000000013_1,"[48, 49, 50, 51, 52, 53, 54]","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","[8, 8, 8, 8, 8, 7, 7]","[3.27, 3.27, 3.27, 3.27, 3.27, 3.27, 3.27]",42,-0.166667
32791,000000000000014_2,"[21, 22, 23, 24, 25, 26, 27]","['Heavy', 'Heavy', 'Heavy', 'Heavy', 'Heavy', ...","[8, 8, 8, 8, 8, 7, 7]","[3.27, 3.27, 3.27, 3.27, 3.27, 3.27, 3.27]",42,-0.166667
